# Computational Drug Discovery Tutorial

Welcome to this hands-on notebook accompanying the blog tutorial.  
We'll cover:

1. **Environment Setup**  
2. **Protein Preparation**  
3. **Ligand Preparation**  
4. **Docking with AutoDock Vina**  
5. **Optional Mini Virtual Screening**  
6. **Basic ML Model**  
7. **Next Steps & Wrap-Up**  

---

## 1. Environment & Initial Setup



In [2]:
import sys
import os
import rdkit
import deepchem
from rdkit import Chem
print("RDKit version:", rdkit.__version__)
print("DeepChem version:", deepchem.__version__)

# Define a data directory (adjust to your structure)
DATA_DIR = "./data"
RESULTS_DIR = "./results"

os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(RESULTS_DIR, exist_ok=True)

print("Data directory:", DATA_DIR)
print("Results directory:", RESULTS_DIR)


RDKit version: 2022.09.5
DeepChem version: 2.8.0


In [3]:
import os
import requests

DATA_DIR = "./data"
os.makedirs(DATA_DIR, exist_ok=True)

# 1) Download the 2R3J PDB directly from RCSB
pdb_id = "2R3J"
pdb_url = f"https://files.rcsb.org/download/{pdb_id}.pdb"
pdb_file_path = os.path.join(DATA_DIR, f"{pdb_id}.pdb")

print(f"Downloading PDB {pdb_id} to {pdb_file_path} ...")
response = requests.get(pdb_url)
with open(pdb_file_path, "wb") as f:
    f.write(response.content)
print("Download complete!")

# 2) Define some known CDK2 inhibitors by SMILES
#    (These can be used in the ligand preparation step)
ligand_smiles_dict = {
    "Roscovitine": "CC(C)c1ccc(cc1NC(=O)C(CC)NC(=O)c1ccc(cc1)C#N)N(C)C",
    "Staurosporine": "CC1C=C(C)C2=C(NC3=NC=CC=CC3=O)C(C)=CC(C)C2=C1NC1=NC=CC=C1=O"
}

print("We have the following ligands to work with:")
for lig_name, smiles in ligand_smiles_dict.items():
    print(f"  - {lig_name}: {smiles}")


Download complete!
We have the following ligands to work with:
  - Roscovitine: CC(C)c1ccc(cc1NC(=O)C(CC)NC(=O)c1ccc(cc1)C#N)N(C)C
  - Staurosporine: CC1C=C(C)C2=C(NC3=NC=CC=CC3=O)C(C)=CC(C)C2=C1NC1=NC=CC=C1=O
